In [4]:
# accounts
import json
import solana
import time
import sys
import glob
from solana.rpc.api import Client
from solana.blockhash import Blockhash
from solana.publickey import PublicKey
from solana.system_program import CreateAccountWithSeedParams, create_account_with_seed, CreateAccountParams, create_account
from solana.transaction import Transaction, AccountMeta, TransactionInstruction
from solana.system_program import SYS_PROGRAM_ID
from spl.token.client import Token
from solana import keypair
from solana.publickey import PublicKey
from solana.sysvar import SYSVAR_RENT_PUBKEY

In [5]:
TOKEN_PROGRAM_ID: PublicKey = PublicKey("TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA")

first_key_in_key_folder = glob.glob("../keys/*")[0]
payer_loaded_account = solana.keypair.Keypair(json.load(open(first_key_in_key_folder))[:32])


http_client = Client("https://api.devnet.solana.com")
payer_public_key = payer_loaded_account.public_key



# todo: change program_id to system program, change data size =0, change lampord for both alice and bob
tx = Transaction()
alice_keypair = solana.keypair.Keypair.generate()
alice_pubkey = alice_keypair.public_key



tx = Transaction()
bob_keypair = solana.keypair.Keypair.generate()
bob_pubkey = bob_keypair.public_key


print(f"Payer Account: {payer_public_key}")
print(f"alice Account: {alice_pubkey}")
print(f"bob Account: {bob_pubkey}")

Payer Account: 763cSApzBT5scjhSd6BdnkKkMYBhiQmPn2gctswkdkJM
alice Account: 3f96k5Y1LogdXaK1iok3tjGwqx4DcKNwE9tk2G7zZRE8
bob Account: 87N1sqyZ9yedPR28gDYv8GvZG2eRUvDkeVwkPdkDomuw


In [6]:
X_mint_account_address = Token.create_mint(conn= http_client, payer = payer_loaded_account, mint_authority = payer_loaded_account.public_key, decimals = 0, program_id = TOKEN_PROGRAM_ID, )
Y_mint_account_address = Token.create_mint(conn= http_client, payer = payer_loaded_account, mint_authority = payer_loaded_account.public_key, decimals = 0, program_id = TOKEN_PROGRAM_ID, )
alice_X_token_account = X_mint_account_address.create_associated_token_account(alice_pubkey)
alice_Y_token_account = Y_mint_account_address.create_associated_token_account(alice_pubkey)
bob_X_token_account = X_mint_account_address.create_associated_token_account(bob_pubkey)
bob_Y_token_account = Y_mint_account_address.create_associated_token_account(bob_pubkey)

In [7]:
X_mint_account_address.mint_to(alice_X_token_account,payer_loaded_account,1000)
Y_mint_account_address.mint_to(bob_Y_token_account,payer_loaded_account,100)

{'jsonrpc': '2.0',
 'result': '4H72Xy3Fx3M4oghFXNcNPeZNKfJ41xWpUbmyqkA5WbJSd3PA4R5k7pPCPZMEw2CM2cTprcxzhJXyJATVPBeJBDYF',
 'id': 45}

In [8]:
import time
http_client.request_airdrop(payer_loaded_account.public_key, 1000000000)
time.sleep(20)
http_client.request_airdrop(alice_pubkey, 1000000000)
time.sleep(20)
http_client.request_airdrop(bob_pubkey, 1000000000)
print(http_client.get_balance(alice_pubkey))
# print(http_client.get_balance(bob_pubkey))
# print(http_client.get_balance(payer_loaded_account.public_key))
# print(http_client.get_minimum_balance_for_rent_exemption(7)["result"])

{'jsonrpc': '2.0', 'result': {'context': {'slot': 86720413}, 'value': 1000000000}, 'id': 49}


In [9]:
# escrow meta-data account
deployed_program_key = glob.glob("../target/deploy/*.json")[0]
deployed_program_key_account = solana.keypair.Keypair(json.load(open(deployed_program_key))[:32])
program_id = deployed_program_key_account.public_key


# tx = Transaction()
# escrow_keypair = solana.keypair.Keypair.generate()
# escrow_pubkey = escrow_keypair.public_key
# data_size = 100
# lamports = http_client.get_minimum_balance_for_rent_exemption(data_size)["result"]
# transaction_instruction = create_account(
#         params=CreateAccountParams(
#             from_pubkey=payer_public_key, new_account_pubkey=escrow_pubkey,
#             lamports=lamports, space=data_size, program_id=program_id
#         )
#     )

# tx = tx.add(transaction_instruction)
# transaction_results = http_client.send_transaction(tx, *[payer_loaded_account,escrow_keypair])
# print(f"Program ID: {program_id}")
# print(f"Escrow pubkey: {escrow_pubkey}")

In [10]:
# todo, creat seed more specific
x_seeds = [
    b"vault_x14tintin",
    bytes(alice_pubkey),
    bytes(bob_pubkey),
    bytes(X_mint_account_address.pubkey),
    bytes(Y_mint_account_address.pubkey),
#     bytes(escrow_pubkey),
]

y_seeds = [
    b"vault_y14tintin",
    bytes(alice_pubkey),
    bytes(bob_pubkey),
    bytes(X_mint_account_address.pubkey),
    bytes(Y_mint_account_address.pubkey),
#     bytes(escrow_pubkey),
]

escrow_seeds = [
    b"escrow14tintin",
    bytes(alice_pubkey),
    bytes(bob_pubkey),
    bytes(X_mint_account_address.pubkey),
    bytes(Y_mint_account_address.pubkey),
#     bytes(escrow_pubkey),
]


vaultx, xseed = PublicKey.find_program_address(seeds=x_seeds,program_id=program_id)
vaulty, yseed = PublicKey.find_program_address(seeds=y_seeds,program_id=program_id)
escrow_address, escrow_seed = PublicKey.find_program_address(seeds=escrow_seeds,program_id=program_id)

In [9]:
print(vaultx)
print(vaulty)
print(escrow_address)
print(program_id)

CGmCX6GBwbXgMV6hedVsq2117poEJ2X3kiujeB6DTd9k
4bjNMkap1gyd4F7tzYKXgdoydXcZjmngCuBd34m6q3MQ
7pprnzMwKCTz2c76BoKefA8P9Yi9vsYcEmMojBWnjuG6
3iGuqM3Loj9xd71anJzHD3caXrW5SDhyuCiKjVALoX5H


In [10]:
# http_client.request_airdrop(escrow_address, 1000000000)
# print(http_client.get_balance(escrow_address))

In [11]:
# initialize transaction
from struct import *
data = pack('<BQQ', 0,13,15)

import struct
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=True),
          AccountMeta(pubkey=X_mint_account_address.pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=Y_mint_account_address.pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=vaultx, is_signer=False, is_writable=True),
          AccountMeta(pubkey=vaulty, is_signer=False, is_writable=True),
          AccountMeta(pubkey=payer_public_key, is_signer=True, is_writable=False),
          AccountMeta(pubkey=alice_pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=bob_pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
          AccountMeta(pubkey=SYSVAR_RENT_PUBKEY, is_signer=False, is_writable=False),
          AccountMeta(pubkey=SYS_PROGRAM_ID, is_signer=False, is_writable=False),
#           AccountMeta(pubkey=program_id, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=  data, #struct.pack("<LHB",1,1,1) # b58encode(bytes([9] * 7))#b'0001000100010001000100010001',
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[payer_loaded_account])#, *[payer_loaded_account])

In [12]:
time.sleep(200)
a = http_client.get_account_info(escrow_address)['result']['value']['data'][0]
import base64
print(unpack('<QQ', base64.b64decode(a)[:16]))
print(unpack('<Q??', base64.b64decode(a)[208:218]))
print(base64.b64decode(a)[16:16+32])
bytes(alice_pubkey)

# Q 8 bytes
# a 400 bytes


# xval: u64,   8 byte
# yval: u64,   8 byte
# a_pub_key: Pubkey, 32 byte
# b_pub_key: Pubkey, 32 byte
# mint_x_pub_key: Pubkey,  32 byte
# mint_y_pub_key: Pubkey,  32 byte
# vault_x_pub_key: Pubkey, 32 byte
# vault_y_pub_key: Pubkey, 32 byte
# init_deposit_status: u64, 8 byte
# is_a_withdrawed: u8,    1byte
# is_b_withdrawed: u8,    1byte

(13, 15)
(1, False, False)
b"\xda\x08\xd8\x10\x19i\xfa\xcb\xa7\xe3\xd5\x99\x00d\x90|\xc5F\x01\xc88'\x0c\xde\xe0\xf2\x89\xf2&\xc1r1"


b"\xda\x08\xd8\x10\x19i\xfa\xcb\xa7\xe3\xd5\x99\x00d\x90|\xc5F\x01\xc88'\x0c\xde\xe0\xf2\x89\xf2&\xc1r1"

In [14]:
# deposite x
from struct import *
data = pack('<B', 1)

import struct
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=True),
          AccountMeta(pubkey=alice_X_token_account, is_signer=False, is_writable=True), # x_a info
          AccountMeta(pubkey=vaultx, is_signer=False, is_writable=True),
          AccountMeta(pubkey=alice_pubkey, is_signer=True, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=  data,
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[alice_keypair])#, *[payer_loaded_account])

In [15]:
# deposite y
time.sleep(30)
from struct import *
data = pack('<B', 1)

import struct
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=True),
          AccountMeta(pubkey=bob_Y_token_account, is_signer=False, is_writable=True), # x_a info
          AccountMeta(pubkey=vaulty, is_signer=False, is_writable=True),
          AccountMeta(pubkey=bob_pubkey, is_signer=True, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=  data,
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[bob_keypair])#, *[payer_loaded_account])

In [ ]:
vaultx

In [ ]:
bob_Y_token_account

In [ ]:
bob_X_token_account

In [ ]:
alice_X_token_account

In [16]:
# withdraw alice
from struct import *
data = pack('<B', 2)

import struct
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=True),
          AccountMeta(pubkey=alice_Y_token_account, is_signer=False, is_writable=True), # x_a info
          AccountMeta(pubkey=vaulty, is_signer=False, is_writable=True),
          AccountMeta(pubkey=alice_pubkey, is_signer=True, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=  data,
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[alice_keypair])#, *[payer_loaded_account])

{'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 0: invalid account data for instruction', 'data': {'accounts': None, 'err': {'InstructionError': [0, 'InvalidAccountData']}, 'logs': ['Program 3iGuqM3Loj9xd71anJzHD3caXrW5SDhyuCiKjVALoX5H invoke [1]', 'Program log: Instruction: Withdrawal', 'Program log: w1', 'Program log: Hey it is not the time to withdraw, you may go ahead and cancel to get back your token!', 'Program 3iGuqM3Loj9xd71anJzHD3caXrW5SDhyuCiKjVALoX5H consumed 3063 of 200000 compute units', 'Program 3iGuqM3Loj9xd71anJzHD3caXrW5SDhyuCiKjVALoX5H failed: invalid account data for instruction']}}


Exception: Failed to send transaction

In [ ]:
# withdraw bob
from struct import *
data = pack('<B', 2)

import struct
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=True),
          AccountMeta(pubkey=bob_X_token_account, is_signer=False, is_writable=True), # x_a info
          AccountMeta(pubkey=vaultx, is_signer=False, is_writable=True),
          AccountMeta(pubkey=bob_pubkey, is_signer=True, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=  data,
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[bob_keypair])#, *[payer_loaded_account])

In [ ]:
# Trouble shooting
# {'code': -32602, 'message': 'invalid transaction: index out of bounds'}: probabily signed with different account in send_transaction function
# debit credit, pobably you need to airdrop solana to the account